In [67]:
import pandas as pd
import numpy as np
import json

# TRATAMENTO DE TEXTOS
import re
import nltk
stopwords=nltk.corpus.stopwords.words('portuguese')


In [68]:
df = pd.read_json(r'reviews_merge.json')
df = df[['title','avaliacao']]
df['avaliacao'] = df['avaliacao'].map({0:'Negativo',1:'Neutro',2:'Positivo', np.nan:'Indefinido'}, na_action=None)
base = [tuple(x) for x in df.to_numpy()]


In [69]:
df.query('avaliacao == "Indefinido"')

Empty DataFrame
Columns: [title, avaliacao]
Index: []

In [95]:
def fazstemmer(frases):
    stemmer = nltk.stem.RSLPStemmer()
    frasessstemming = []
    for (palavras, emocao) in frases:
        comstemming = [str(stemmer.stem(p))
                       for p in palavras.split() if p not in stopwords]
        frasessstemming.append((comstemming, emocao))
    return frasessstemming
 
 
frasescomstemming = fazstemmer(base)
#print(frasescomstemming)

In [71]:
def buscapalavras(frases):
    todaspalavras = []
    for (palavras, emocao) in frases:
        todaspalavras.extend(palavras)
    return todaspalavras
 
todaspalavras = buscapalavras(frasescomstemming)
#print(todaspalavras)

In [72]:
def buscafrequencia(palavras):
    palavras = nltk.FreqDist(palavras)
    return palavras
frequencia = buscafrequencia(todaspalavras)


In [105]:
def busca_palavrasunicas(frequencia):
    freq = frequencia.keys()
    return freq
 
palavrasunicas = busca_palavrasunicas(frequencia)
print(palavrasunicas)

dict_keys(['empr', 'dinâm', 'agrad', 'trabalhar.', 'não', 'perspec', 'cresc', 'carre', 'foi', 'boa', 'experienc', 'trabalhar,', 'oportun', 'funcion', 'respeit', 'plan', 'carg', 'salári', 'risc', '.', 'excel', 'pra', 'trabalh', 'ótim', 'fabric', 'aviões.', 'pont', 'alt', 'tecnologia.', 'pesso', 'educ', 'dinâmica,', 'espirit', 'equip', 'ambi', 'bom', 'sensac', 'bem', 'estrur', 'funcionari', 'trabalho.', 'inst', 'e.pr', 'aplic', 'própri', 'principi', 'melh', 'contínu', 'maravilh', 'educadas,', 'cult', '5', 'estrelas.', 'muit', 'pessoas:', 'problemas.', 'seria:', 'gest', 'problem', 'estrutura.', 'empresa,', 'recom', 'qualqu', 'interess', 'benefícios,', 'colabor', 'sér', 'satisfatór', 'valor', ',', 'lug', 'uma', 'ampl', 'conhec', 'ges', 'sup', 'dedic', 'ensin', 'funcionários.', 'bast', 'preocup', 'funcionários!', 'empre', 'ano', '80', 'ness', 'profiss', 'tud', 'aparent', 'fantas', 'recomendo.', 'exel', 'otím', 'estrut', 'empresa.', 'dificil', 'faz', 'avaliação,', 'poi', 'embra', '37', 'conc

In [74]:
def extrai_palavras(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavrasunicas:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [75]:
basecompleta = nltk.classify.apply_features(extrai_palavras,frasescomstemming)

In [76]:
classificador = nltk.NaiveBayesClassifier.train(basecompleta) 

In [103]:
frases_Novas = ['ruim','Boa']
stemmer = nltk.stem.RSLPStemmer()
frase_resultado =[]
for i in frases_Novas:
    frase= i
    frase_i = []
    for (palavras) in frase.split():
        comstem = [p for p in palavras.split()]
        frase_i.append(str(stemmer.stem(comstem[0])))
    frase_resultado.append(frase_i)
print(frase_resultado)

[['ruim'], ['boa']]


In [104]:
for i in frase_resultado:
    nova_frase = extrai_palavras(i)
 
    distribuicao = classificador.prob_classify(nova_frase)
    print('-----------------------')
    for classe in distribuicao.samples():
        print("%s: %f" % (classe, distribuicao.prob(classe)))

-----------------------
Positivo: 0.990555
Negativo: 0.009431
Neutro: 0.000014
-----------------------
Positivo: 0.999934
Negativo: 0.000066
Neutro: 0.000000
